In [9]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../data/fixtures.csv')

# EDA and Data Cleaning

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4930 entries, 0 to 4929
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fixture.id              4930 non-null   int64  
 1   fixture.referee         3138 non-null   object 
 2   fixture.timezone        4930 non-null   object 
 3   fixture.date            4930 non-null   object 
 4   fixture.timestamp       4930 non-null   int64  
 5   fixture.periods.first   4560 non-null   float64
 6   fixture.periods.second  4560 non-null   float64
 7   fixture.venue.id        2556 non-null   float64
 8   fixture.venue.name      4930 non-null   object 
 9   fixture.venue.city      3100 non-null   object 
 10  fixture.status.long     4930 non-null   object 
 11  fixture.status.short    4930 non-null   object 
 12  fixture.status.elapsed  4930 non-null   int64  
 13  league.id               4930 non-null   int64  
 14  league.name             4930 non-null   

## Remove redundant columns (initial)

In [14]:
## Check for unqiue values
print(data["fixture.status.short"].value_counts())
print(data["fixture.status.long"].value_counts())
print(data["fixture.status.elapsed"].value_counts())


fixture.status.short
FT    4930
Name: count, dtype: int64
fixture.status.long
Match Finished    4930
Name: count, dtype: int64
fixture.status.elapsed
90    4930
Name: count, dtype: int64


In [18]:
clean_data = data.drop(columns=["fixture.id", "fixture.timezone", "fixture.timestamp", 
                                "fixture.periods.first", "fixture.periods.second", 
                                "fixture.venue.name", "fixture.venue.city", 
                                "fixture.status.long", "fixture.status.short", 
                                "fixture.status.elapsed", "league.id", "league.name",
                                "league.country", "league.logo", "league.flag",
                                "teams.home.name", "teams.home.logo",
                                "teams.away.name", "teams.away.logo",
                                "goals.home", "goals.away",
                                "score.extratime.home", "score.extratime.away",
                                "score.penalty.home", "score.penalty.away",])

### Reasons for dropping columns

* `fixture.*`
  * `id` is simply row index from api
  * `timezone` info related to time is relative. Should not be affected by timezone conversion.
  * `timestamp`, `periods.first`, `periods.second` derived from `date`
  * `venue.name`, `venue.city` represented by `venue.id` 
  * `fixture.status.long`, `fixture.status.short`, `fixture.status.elapsed` contains only 1   value. No info gain

* `league.*`
  * `id`, `name`, `country`, `logo`, `flag` constant

* `teams.*`
  * `home.name`, `home.logo`, `away.name`, `away.logo` represented by `[home|away].id`

* `goals.*` same as `score.fulltime.*`
  
* `score.*`
  * `extratime.*`, `penalty.*` always None. No such thing in a leauge game


## Cleaning the data

In [24]:
## Rename columns
clean_data = clean_data.rename(columns={"fixture.referee": "referee",
                                        "fixture.date": "date",
                                        "fixture.venue.id": "venue_id",
                                        "league.season": "season",
                                        "league.round": "round",
                                        "teams.home.id": "home_id",
                                        "teams.home.winner": "home_winner",
                                        "teams.away.winner": "away_winner",
                                        "teams.away.id": "away_id",
                                        "score.halftime.home": "ht_home",
                                        "score.halftime.away": "ht_away",
                                        "score.fulltime.home": "ft_home",
                                        "score.fulltime.away": "ft_away",
                                        })

## Convert date to datetime
clean_data["date"] = pd.to_datetime(clean_data["date"])

In [25]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4930 entries, 0 to 4929
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   referee      3138 non-null   object             
 1   date         4930 non-null   datetime64[ns, UTC]
 2   venue_id     2556 non-null   float64            
 3   season       4930 non-null   int64              
 4   round        4930 non-null   object             
 5   home_id      4930 non-null   int64              
 6   home_winner  3738 non-null   object             
 7   away_id      4930 non-null   int64              
 8   away_winner  3738 non-null   object             
 9   ht_home      4930 non-null   int64              
 10  ht_away      4930 non-null   int64              
 11  ft_home      4930 non-null   int64              
 12  ft_away      4930 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(1), int64(7), object(4)
memory usage: 500.8

In [23]:
clean_data

,referee,date,venue_id,season,round,home_id,home_winner,away_id,away_winner,ht_home,ht_away,ft_home,ft_away
0,NaN,2010-08-14T11:45:00+00:00,NaN,2010,Regular Season - 38,47,NaN,50,NaN,0,0,0,0
1,NaN,2010-08-14T14:00:00+00:00,NaN,2010,Regular Season - 38,66,True,48,False,2,0,3,0
2,NaN,2010-08-14T14:00:00+00:00,NaN,2010,Regular Season - 38,67,True,45,False,1,0,1,0
3,NaN,2010-08-14T14:00:00+00:00,NaN,2010,Regular Season - 38,61,False,1356,True,0,3,0,4
4,NaN,2010-08-14T14:00:00+00:00,NaN,2010,Regular Season - 38,68,NaN,36,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4925,S. Attwell,2023-05-28T15:30:00+00:00,8560.0,2022,Regular Season - 38,45,True,35,False,0,0,1,0
4926,A. Taylor,2023-05-28T15:30:00+00:00,546.0,2022,Regular Season - 38,63,False,47,True,0,1,1,4
4927,S. Hooper,2023-05-28T15:30:00+00:00,547.0,2022,Regular Season - 38,46,True,48,False,1,0,2,1
4928,R. Jones,2023-05-28T15:30:00+00:00,556.0,2022,Regular Season - 38,33,True,36,False,1,1,2,1
